#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [1]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [2]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [3]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [4]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> str:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is optional. You can pass 'None' and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story.__str__()
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result.__str__()

@tool
def update_story(updated_story: dict, runtime: ToolRuntime) -> str:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json. it should contain all the fields of a Story object.
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "story successfully updated. the new story: {}".format(validated_story.__str__())
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "updating story failed"
    


##### Agent definition

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

gpt_llm = ChatOllama(
    model="gpt-oss:20b", 
    reasoning="high"
)

my_agent = create_agent(
    gpt_llm,
    tools=[read_or_create_story, update_story],
    checkpointer=InMemorySaver(), # this retains the conversation history
    system_prompt="""
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """,
    response_format=ToolStrategy(Story) #TODO: this might get in the way of generating other structured data. Other than Story.
)


##### Running the agent

In [15]:
bool()

False

In [28]:
from langchain.messages import AIMessage, AIMessageChunk
from IPython.display import Markdown, display, clear_output
from typing import Union, Literal

len_messages_before_model_call = 1

type_of_content: Union[Literal['reasoning'], Literal['text'], None] = 'reasoning'
most_recent_message_type: Union[Literal['reasoning'], Literal['text'], None] = 'reasoning'
response_text = ''

while True:
    
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    # response = my_agent.invoke(
    #         {'messages': [{"role": "user", "content": user_input}]},
    #         {'configurable': {"thread_id": 1}},
    #     )
    
    # latest_messages = response['messages'][len_messages_before_model_call:]
    # for latest_message in latest_messages:
    #     latest_message_content = latest_message.content
    #     display(Markdown(latest_message_content))
    
    # len_messages_before_model_call = len(response['messages'])
    # ---
    for chunk in my_agent.stream(
        {'messages': [{'role': 'user', 'content': user_input}]}, 
        {'configurable': {'thread_id': 1}}, 
        # stream_mode='updates'
        stream_mode='messages'
        ):
        # print('chunk {}'.format(chunk))
        current_message = chunk[0] # type: ignore
        # print('first message: {}'.format(current_message))
                    
        # Determine the type of current response whether it is a reasoning chunk or a chunk of an actual response.
        if len(current_message.content) > 0:
            type_of_content = 'text'
            if most_recent_message_type != 'text':
                response_text += '\n\nAnswer:\n\n'
                most_recent_message_type = 'text'
        elif len(current_message.content) == 0 and bool(current_message.additional_kwargs.get('reasoning_content', '')):
            type_of_content = 'reasoning'
            if most_recent_message_type != 'reasoning':
                response_text += "\n\nReasoning:\n\n"
                most_recent_message_type = 'reasoning'
        else:
            type_of_content = 'text'
            
        # Depending on the content type, get the relevant field and print it out
        if type_of_content == 'text':
            content_blocks_of_current_message = current_message.content_blocks
            if len(content_blocks_of_current_message) > 0 and 'text' in content_blocks_of_current_message[0]:
                current_text = content_blocks_of_current_message[0].get('text', '')
                # print(current_text, end="", flush=True) 
                response_text += current_text
        
        elif type_of_content == 'reasoning':
            reasoning_of_current_message = current_message.additional_kwargs.get('reasoning_content', '')
            if len(reasoning_of_current_message) > 0:
                # print(reasoning_of_current_message, end="", flush=True)
                response_text += reasoning_of_current_message

        else:
            print("the type is not AIMessage or AIMessageChunk, but {}".format(type(current_message)))   
            
        clear_output()
        display(Markdown(response_text))
        
    print("\n")
    

        


The user has said: "i want to write a story". Then repeated that phrase multiple times. The system instructions say: "You are an expert screenwriter. ... Do not update something before I confirm it. Do not make up ids. Tell what you did, even if you only updated something. Always go through the story json fields one by one and do not try to answer more than one field at a time. Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms. Do not sugercoat anything or do not try to make it look pleasing."

We must respond following those rules. The user repeated "i want to write a story" but no genre was specified. We should ask them to specify the genre. But earlier the user had "sitcom". Did the user mention sitcom? Yes, the assistant responded with "Genre - You’d like the story to be a sitcom". The user says "i want to write a story" again. So maybe they want to confirm. According to the instructions, we must not update something before the user confirms. The user didn't confirm the genre. The user says "i want to write a story" but didn't say which genre. They previously said "sitcom". The assistant responded that they'd like the story to be a sitcom. The user didn't confirm. We should ask for confirmation. The instructions: "Do not update something before I confirm it." So we should not update the genre until the user says so. So we should ask them: "Please confirm that you want the genre to be sitcom." Then wait. Also "Always go through the story json fields one by one and do not try to answer more than one field at a time." We have only processed the genre field. So we can proceed with "genre" field.

Thus we should respond with something like: "Please confirm the genre. Are we setting the genre to 'sitcom'?" Also "I am waiting for your confirmation." We'll do that.

Answer:

**Genre** –  
I’d like to set the `genre` field to “sitcom”.  
Please confirm (or suggest another genre) before I make the change.

Reasoning:

We need to update the story with genre "sitcom". This is a field update. According to instructions: "Always go through the story json fields one by one and do not try to answer more than one field at a time. Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms." We need to update one field: genre. Then after updating, we might want to discuss the logline or other fields? But instructions say: "Do not update something before I confirm it. Do not make up ids. Tell what you did, even if you only updated something." So we should respond with an update: we updated genre. Also mention that we left other fields "#toBeDefined". Also give some comments about sitcom genre. But we must not do multiple fields at once. We only updated genre. So we should respond: updated genre field. Then maybe ask for next field, or give suggestions? We can give suggestions but not change fields. We'll give some critiques/ideas about sitcom.

We must call update_story function. But we only need to update genre. The function expects updated_story containing all fields. The instructions: "Overwrites the existing outdated story with the given updated_story. Args: updated_story: contains the new version of story as a json. it should contain all the fields of a Story object." So we must supply all fields: character_ids, genre, id, logline, summary, plotPoints, season_ids. We'll set the others to current values: They are "#toBeDefined" except id. But we can pass them. That will override. So we should get current story, update genre to "sitcom", and keep other fields same. But we need to have the current story. We have ID: f6860894-dbf2-4e63-aa9e-a659f85794dd. We can fetch the story? There's no read function; but we can approximate. We know current values: id is that, genre is now "sitcom", logline "#toBeDefined", summary "#toBeDefined", plotPoints [], character_ids [], season_ids []. So we send updated_story with those values.

But instructions: "Always go through the story json fields one by one and do not try to answer more than one field at a time." But we are updating only one field. But we must provide updated_story with all fields. It's okay. We need to mention we updated the genre.

After that, we might give some feedback on sitcom genre: what are typical pitfalls, tropes, successes.

We should not make up new ids. We'll use same id.

We should also not say "I updated the genre field" as final answer. We should respond that we updated the genre, and maybe ask for next step: logline. But also give some suggestions. The instruction says "Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms. Do not sugercoat anything or do not try to make it look pleasing." So we should give honest critique of sitcom as a genre for a story: it's usually episodic, comedic, set in a common environment, etc. Might mention pitfalls: rely on canned jokes, lack depth, etc. Suggest ways to differentiate: maybe set in a unique workplace, add a twist, etc.

We must not ask any question. We can ask: "Would you like to work on the logline next?" It's okay to ask for next field. But instructions: "Always go through the story json fields one by one and do not try to answer more than one field at a time." We are at genre. We updated it. Next we can propose to fill logline. But maybe we should ask. But we can also give suggestions: "Now that we've set the genre to sitcom, think about what comedic premise you want." That may be okay. It is not answering multiple fields. It's just advice.

So final: mention we updated genre, give feedback, ask for next step or logline. But not multiple field updates.

Thus, I'll produce a response: mention we updated the genre to sitcom, mention that the story object is now updated, and give critique on sitcom.

Then maybe prompt: "What logline would you like?" This is a question. That is fine.

We need to call the update_story function. Use the function call. Then respond.

We need to provide the updated_story object: with id, genre "sitcom", other fields unchanged.

So function call:

{
  "updated_story": {
    "id": "f6860894-dbf2-4e63-aa9e-a659f85794dd",
    "genre": "sitcom",
    "logline": "#toBeDefined",
    "summary": "#toBeDefined",
    "plotPoints": [],
    "character_ids": [],
    "season_ids": []
  }
}

We need to use correct field names: In Story, the fields are: character_ids, genre, id, logline, summary, plotPoints, season_ids. Yes.

Thus, function call.

Then final message: "I updated the genre field to 'sitcom'...".

We also provide critique.

Ok, proceed.

Answer:

story successfully updated. the new story: id='f6860894-dbf2-4e63-aa9e-a659f85794dd' genre='sitcom' logline='#toBeDefined' summary='#toBeDefined' plotPoints=[] character_ids=[] season_ids=[]

Reasoning:

The user said: "yes, set it to sitcom". The assistant updated the story. The story now has id, genre set to 'sitcom', logline and summary still '#toBeDefined', plotPoints empty, character_ids empty, season_ids empty.

Now what should the assistant do? According to instructions, the assistant should continue to ask the next field one by one. We have the story json fields: character_ids, genre, logline, summary, plotPoints, season_ids. The genre is set. The next field to fill is logline. So we should say something like: "Now let's define the logline." But we need to ask the user for input or propose a suggestion. The instructions say: "Always go through the story json fields one by one and do not try to answer more than one field at a time." So we should ask about the next field: logline.

We must provide new ideas and critics to the user's ideas and decisions, but we haven't gotten an idea from the user except genre. We can ask them to provide a logline or propose one. We should also perhaps give feedback on their choice of genre (sitcom). So we can comment on sitcoms, maybe mention common tropes, pitfalls, etc. But we need to only focus on the logline field next. The instructions: "Tell what you did, even if you only updated something. Provide new ideas and critics to the user's ideas and decisions." So we can provide critiques of sitcoms. But also we should ask the user to provide a logline. So we can ask: "Now, what would be the logline for this sitcom?" Or we can propose a logline and ask for feedback.

We should not answer more than one field at a time. So we will not ask for summary or plot points. Just ask about logline.

Thus, the assistant should respond: "I set the genre to sitcom. Next, we need a logline. Provide one or let me suggest." Provide some critique on sitcoms. Then ask.

Let's draft:

"Genre set to sitcom. Sitcoms often rely on recurring characters and situational humor. Common pitfalls: relying too heavily on situational cliches, not enough character development, lack of fresh conflict. You might want to consider a unique setting or unusual character dynamic. Now, what is the logline for your sitcom?" This satisfies the instruction.

Also, mention that we are going to fill the next field logline. So we can ask for user input. The user can provide a logline. If they don't, we can propose one. But we should not answer the next fields. So the best approach: ask user for logline. Also provide suggestions and critique.

Thus, let's answer accordingly.

Answer:

**What I’ve done so far**

I set the story’s **genre** to **“sitcom.”**  

**Sitcom‑specific thoughts**

- Sitcoms thrive on *recurring characters* and *situational humor* that can be stretched across multiple episodes.  
- Common pitfalls include falling into too‑predictable tropes (e.g., “the clueless roommate,” “the over‑the‑top boss”) or stalling the narrative with jokes that never advance character or plot.  
- Freshness often comes from an unusual setting (a high‑school cafeteria that’s actually a space‑station lounge, a coffee shop where the barista is a time‑traveler, etc.) or from giving the main characters *subtle, evolving motivations* that go beyond the one‑liner punchline.

---

### Next step: **Logline**

The logline should be a one‑to‑two‑sentence hook that tells the *central premise* and the *conflict* that keeps the show going. It’s the “pitch” you’d give to a network exec or a friend who’s never seen it.

**Examples of good sitcom loglines**

1. *“When a group of mismatched interns at a failing law firm try to save the office from a hostile takeover, they discover that the real law of the land is the one they make themselves.”*  
2. *“A socially awkward grad student becomes the unlikely host of a late‑night talk show, only to find that his bizarre guests are all hiding a very serious secret.”*

**Your turn**

Could you give me a logline for your sitcom? If you’d like, I can help you craft one.